Object detection (via Detectron2).

Optical character recognition (OCR) (via Tesseract and PyTesseract).

# Multi-Type-TD-TSR Demo Notebook
## 1. Install all the prerequisite


In [1]:
# install dependencies:
!pip install pyyaml
!pip install torch
!pip install torchvision

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

!sudo apt install tesseract-ocr
!pip install pytesseract

import torch, torchvision
torch.__version__


2.5.1+cu121 True
gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/unive

'2.5.1+cu121'

In [2]:
import time
start_time=time.time()

In [3]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-epbf__uj
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-epbf__uj
  Resolved https://github.com/facebookresearch/detectron2.git to commit c69939aa85460e8135f40bce908a6cddaa73065f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.4 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [4]:
end_time=time.time()
print("Time taken for loading the modules",end_time-start_time)

Time taken for loading the modules 281.64215421676636


In [5]:
start_time_repo=time.time()

## 2. Clone the Repository

In [6]:
# !git clone https://github.com/Psarpei/Multi-Type-TD-TSR.git
# !mv Multi-Type-TD-TSR/ Multi_Type_TD_TSR

## 3. Download Table Detection Weights and Configuration File

In [7]:
!gdown --id 11FgFTy0MyVUMGd00T_InEDaarB4qAlP8
!gdown --id 1WBk6kHHyvyEzoPBsRr2BvFY51zURjd4R
!gdown --id 1PfA2uws919gc893-x9uMIz06zWEko8nF

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=11FgFTy0MyVUMGd00T_InEDaarB4qAlP8
From (redirected): https://drive.google.com/uc?id=11FgFTy0MyVUMGd00T_InEDaarB4qAlP8&confirm=t&uuid=f3f0eee7-853d-4fe6-b291-1e5fb615f4ac
To: /content/model_final.pth
100% 1.10G/1.10G [00:31<00:00, 34.6MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1WBk6kHHyvyEzoPBsRr2BvFY51zURjd4R
To: /content/All_X152.yaml
100% 534/534 [00:00<00:00, 2.32MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecate

In [8]:
clone_time_repo=time.time()
print("Time taken to clone the repo and download the weights",clone_time_repo-start_time_repo)

Time taken to clone the repo and download the weights 46.289571046829224


## 4. Import Everything

In [9]:
import detectron2
import deskew as deskew
import table_detection as table_detection
import table_structure_recognition_all as tsra
import table_xml as txml
import table_ocr as tocr
import pandas as pd
import os
import json
import itertools
import random
from detectron2.utils.logger import setup_logger
# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog
from google.colab.patches import cv2_imshow
setup_logger()

<Logger detectron2 (DEBUG)>

In [10]:
# prompt: made some changes to this file so. ned to cat to see the content /content/Multi_Type_TD_TSR/google_colab/table_structure_recognition_all.py

!cat table_structure_recognition_all.py

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv


from google.colab.patches import cv2_imshow

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract as tess
import pytesseract


def recognize_structure(img):
    #tess.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

    #print(img.shape)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_height, img_width = img.shape

    #print("img_height", img_height, "img_width", img_width)
    #cv2_imshow(img,)

    # thresholding the image to a binary image
    thresh, img_bin = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY)
    #thresh, img_bin = cv2.threshold(img, 160, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    #img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,5,5)

    #cv2_imshow(img_bin)

    contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #in

## 5. Initialize Table Detection Detector

In [11]:
#create detectron config
cfg = get_cfg()

#set yaml
cfg.merge_from_file('/content/All_X152.yaml')

#set model weights
cfg.MODEL.WEIGHTS = '/content/model_final.pth' # Set path model .pth

predictor = DefaultPredictor(cfg)

[12/09 13:03:36 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/model_final.pth ...


/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

## Testing Here

In [ ]:
document_img = cv2.imread("image2.jpeg")
table_detection.plot_prediction(document_img, predictor)


In [ ]:
table_list, table_coords = table_detection.make_prediction(document_img, predictor)

In [ ]:
list_table_boxes = []

for table in table_list:
    finalboxes, output_img = tsra.recognize_structure(table)
    list_table_boxes.append(finalboxes)

In [ ]:
txml.output_to_xml(table_coords, list_table_boxes)

In [ ]:
tocr.output_to_csv(list_table_boxes[0], output_img)

In [ ]:
!pip install pdf2image


In [ ]:
!pip install poppler-utils

## FOR PDF

In [26]:
import importlib
import table_xml as txml  # Replace with the actual module name
import table_ocr as tocr
# Reload the module to reflect changes
importlib.reload(txml)
importlib.reload(tocr)

<module 'table_ocr' from '/content/table_ocr.py'>

In [13]:
!rm -rf __pycache__

In [14]:
!pip install pdf2image

In [15]:
!apt-get install poppler-utils
import os
os.environ['PATH'] += os.pathsep + '/usr/bin'
# This line assumes pdfinfo is located in /usr/bin.
# Adjust the path if it's different on your system.

from pdf2image import convert_from_path
import cv2
import os


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (139 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123679 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [16]:
from pdf2image import convert_from_path
import cv2
import os

In [17]:
def pdf_to_images(pdf_path, output_folder="pdf_pages", dpi=300):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Convert PDF to list of images
    images = convert_from_path(pdf_path, dpi=dpi)

    image_paths = []
    for idx, image in enumerate(images):
        image_path = os.path.join(output_folder, f"page_{idx + 1}.jpg")
        image.save(image_path, "JPEG")
        image_paths.append(image_path)

    return image_paths

In [18]:
!apt-get install poppler-utils
import os
os.environ['PATH'] += os.pathsep + '/usr/bin'
# This line assumes pdfinfo is located in /usr/bin.
# Adjust the path if it's different on your system.

from pdf2image import convert_from_path
import cv2
import os


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [19]:
pdf_path = "/content/2407.01219v1.pdf"


In [20]:
def process_pdf_images(pdf_path, predictor, tsra, txml, tocr):
    start_time_pdf2img = time.time()
    image_paths = pdf_to_images(pdf_path)  # Convert PDF to images
    end_time_pdf2img = time.time()
    pdf2image_time = end_time_pdf2img - start_time_pdf2img

    results_dir = "results"
    os.makedirs(results_dir, exist_ok=True)

    # Initialize timers for each stage
    time_make_prediction = 0
    time_recognize_structure = 0
    time_csv = 0
    time_xml = 0

    for idx, img_path in enumerate(image_paths):
        try:
            document_img = cv2.imread(img_path)
            start_time_make_prediction = time.time()
            table_list, table_coords = table_detection.make_prediction(document_img, predictor)
            end_time_make_prediction = time.time()
            time_make_prediction += end_time_make_prediction - start_time_make_prediction

            list_table_boxes = []

            for table_idx, table in enumerate(table_list):
                try:
                    start_time_recognize_structure = time.time()
                    finalboxes, output_img = tsra.recognize_structure(table)
                    end_time_recognize_structure = time.time()
                    time_recognize_structure += end_time_recognize_structure - start_time_recognize_structure

                    list_table_boxes.append(finalboxes)

                    # Generate unique file names for each table
                    xml_output_path = os.path.join(results_dir, f"image_{idx + 1}_table_{table_idx + 1}.xml")
                    csv_output_path = os.path.join(results_dir, f"image_{idx + 1}_table_{table_idx + 1}.xlsx")

                    # Save outputs for this table
                    start_time_xml = time.time()
                    txml.output_to_xml([table_coords[table_idx]], [finalboxes], xml_output_path)
                    end_time_xml = time.time()
                    time_xml += end_time_xml - start_time_xml

                    start_time_csv = time.time()
                    tocr.output_to_csv(finalboxes, output_img, csv_output_path)
                    end_time_csv = time.time()
                    time_csv += end_time_csv - start_time_csv
                except Exception as e:
                    print(f"Error processing table {table_idx + 1} in image {idx + 1}: {e}")
                    continue  # Skip to the next table

            print(f"Processed image {idx + 1}/{len(image_paths)} with {len(table_list)} tables.")
        except Exception as e:
            print(f"Error processing image {idx + 1}: {e}")
            continue  # Skip to the next image

    # Summary of processing times
    print("All images processed.")
    print(f"Time taken for PDF to images: {pdf2image_time:.2f} seconds")
    print(f"Time taken for table detection (make_prediction): {time_make_prediction:.2f} seconds")
    print(f"Time taken for table structure recognition: {time_recognize_structure:.2f} seconds")
    print(f"Time taken for CSV generation: {time_csv:.2f} seconds")
    print(f"Time taken for XML generation: {time_xml:.2f} seconds")
    total_time = pdf2image_time + time_make_prediction + time_recognize_structure + time_csv + time_xml
    print(f"Total processing time: {total_time:.2f} seconds")

In [21]:
start_pdf=time.time()

In [22]:
process_pdf_images(pdf_path,predictor,tsra,txml,tocr)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Processed image 1/22 with 1 tables.
Processed image 2/22 with 0 tables.
Processed image 3/22 with 0 tables.
Processed image 4/22 with 1 tables.
Processed image 5/22 with 2 tables.
Error processing table 1 in image 6: list index out of range
Processed image 6/22 with 2 tables.
Error processing table 1 in image 7: list index out of range
Processed image 7/22 with 2 tables.
Processed image 8/22 with 2 tables.
Processed image 9/22 with 1 tables.
Processed image 10/22 with 0 tables.
Processed image 11/22 with 1 tables.
Processed image 12/22 with 0 tables.
Error processing table 1 in image 13: list index out of range
Processed image 13/22 with 1 tables.
Processed image 14/22 with 0 tables.
Processed image 15/22 with 0 tables.
Processed image 16/22 with 0 tables.
Processed image 17/22 with 0 tables.
Processed image 18/22 with 0 tables.
Processed image 19/22 with 0 tables.
Processed image 20/22 with 1 tables.
Processed image 21/22 with 2 tables.
Error processing table 1 in image 22: list index

In [27]:
process_pdf_images(pdf_path,predictor,tsra,txml,tocr)

Processed image 1/22 with 1 tables.
Processed image 2/22 with 0 tables.
Processed image 3/22 with 0 tables.
Processed image 4/22 with 1 tables.
Processed image 5/22 with 2 tables.
Error processing table 1 in image 6: list index out of range
Processed image 6/22 with 2 tables.
Error processing table 1 in image 7: list index out of range
Processed image 7/22 with 2 tables.
Processed image 8/22 with 2 tables.
Processed image 9/22 with 1 tables.
Processed image 10/22 with 0 tables.
Processed image 11/22 with 1 tables.
Processed image 12/22 with 0 tables.
Error processing table 1 in image 13: list index out of range
Processed image 13/22 with 1 tables.
Processed image 14/22 with 0 tables.
Processed image 15/22 with 0 tables.
Processed image 16/22 with 0 tables.
Processed image 17/22 with 0 tables.
Processed image 18/22 with 0 tables.
Processed image 19/22 with 0 tables.
Processed image 20/22 with 1 tables.
Processed image 21/22 with 2 tables.
Error processing table 1 in image 22: list index

In [23]:
end_time_pdf=time.time()

In [24]:
print("Time taken",end_time_pdf-start_pdf)

Time taken 247.7264633178711


In [ ]:
pdf_path_2 = "/content/VERIZON_2022_10K.pdf"
start_pdf=time.time()
process_pdf_images(pdf_path_2,predictor,tsra,txml,tocr)
end_time_pdf=time.time()
print("Time taken",end_time_pdf-start_pdf)

Processed image 1/124 with 3 tables.
Error processing table 1 in image 2: list index out of range
Error processing table 4 in image 2: list index out of range
Processed image 2/124 with 4 tables.
Processed image 3/124 with 1 tables.
Processed image 4/124 with 0 tables.
Processed image 5/124 with 0 tables.
Processed image 6/124 with 0 tables.
Processed image 7/124 with 0 tables.
Processed image 8/124 with 0 tables.
Processed image 9/124 with 0 tables.
Processed image 10/124 with 0 tables.
Processed image 11/124 with 0 tables.
Processed image 12/124 with 0 tables.
Processed image 13/124 with 0 tables.
Processed image 14/124 with 0 tables.
Processed image 15/124 with 0 tables.
Processed image 16/124 with 0 tables.
Processed image 17/124 with 0 tables.
Processed image 18/124 with 0 tables.
Processed image 19/124 with 1 tables.
Processed image 20/124 with 2 tables.
Processed image 21/124 with 0 tables.
Processed image 22/124 with 0 tables.
Processed image 23/124 with 1 tables.
Processed ima

In [ ]:
end_time_pdf=time.time()
print("Time taken",end_time_pdf-start_pdf)

Time taken 1960.200466632843


In [25]:
import shutil
from google.colab import files

# Path to the folder you want to compress and download
folder_path = '/content/results'
zip_filename = '/content/results.zip'

# Compress the folder into a zip file
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', folder_path)

# Download the zip file
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

shutil.rmtree('/content/results', ignore_errors=True)
shutil.rmtree('/content/pdf_pages', ignore_errors=True)
